In [1]:
# Define model
def DefineModel(Model):
    Model.SelectWidth = pyo.Var(Model.Item, Model.Size, domain = pyo.Binary, initialize = 0)   # For each product size, select one of the item widths
    Model.SelectLength = pyo.Var(Model.Item, Model.Size, domain = pyo.Binary, initialize = 0)   # For each product size, select one of the item lengths
    Model.Allocation = pyo.Var(Model.Item, Model.Size, within = pyo.Binary, initialize = 0)   # Allocate each item to one of the products
    
    def rule_LBWidth(Model, i):   # Width of allocated product must be at least width of each item it is allocated to
        return sum(Model.Allocation[i, s] * sum(Model.Width[i] * Model.SelectWidth[i, s] for i in Model.Item) for s in Model.Size) >= Model.Width[i]
    Model.MinWidth = pyo.Constraint(Model.Item, rule = rule_LBWidth)

    def rule_LBLength(Model, i):   # Length of allocated product must be at least width of each item it is allocated to
        return sum(Model.Allocation[i, s] * sum(Model.Length[i] * Model.SelectLength[i, s] for i in Model.Item) for s in Model.Size) >= Model.Length[i]
    Model.MinLength = pyo.Constraint(Model.Item, rule = rule_LBLength)
    
    def rule_once(Model, i):   # Each item is allocated to exactly one product
        return sum(Model.Allocation[i, s] for s in Model.Size) == 1
    Model.AllocateOnce = pyo.Constraint(Model.Item, rule = rule_once)

    def rule_OneW(Model, s):   # Each product has exactly one width
        return sum(Model.SelectWidth[i, s] for i in Model.Item) == 1
    Model.SelectW = pyo.Constraint(Model.Size, rule = rule_OneW)

    def rule_OneL(Model, s):   # Each product has exactly one length
        return sum(Model.SelectLength[i, s] for i in Model.Item) == 1
    Model.SelectL = pyo.Constraint(Model.Size, rule = rule_OneL)
    
    def rule_Obj(Model):   # Minimize waste = Area of allocated product minus area of item, in total for all items
        return sum(sum(Model.Allocation[i, s] * (sum(Model.Width[i] * Model.SelectWidth[i, s] for i in Model.Item) \
               * sum(Model.Length[i] * Model.SelectLength[i, s] for i in Model.Item)) * Model.Weight[i] for s in Model.Size) for i in Model.Item) \
               - sum(Model.Width[i] * Model.Length[i] * Model.Weight[i] for i in Model.Item)
    Model.Obj = pyo.Objective(rule = rule_Obj, sense = pyo.minimize)